In [365]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Generate the iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Build a model using the default setting of fully developing the tree
tree = DecisionTreeClassifier(random_state=0)

# Fit the classifier on training set
tree.fit(X_train, y_train)

# Make the predictions on test set
predictions = tree.predict(X_test)
print("Test set predictions: {}".format(predictions))

# Evaluate the model
accuracy = tree.score(X_test, y_test)
print("Test set accuracy: {:.2f}".format(accuracy))

Test set predictions: [0 1 1 1 0 1 2 2 2 2 1 2 1 1 0 0 0 1 0 1 2 1 1 1 2 1 0 2 0 2 2 2 0 0 0 0 2
 1]
Test set accuracy: 0.92


In [421]:
# imports
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpld3
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, svm
from matplotlib import style

# configs
mpld3.enable_notebook()
warnings.filterwarnings("ignore")

# read csvs & build dataframe
df = pd.DataFrame()
df_temp = pd.read_csv('./datasets/temperatura_featured.csv')
df_vendas = pd.read_csv('./datasets/vendas.csv', ';')
df_conc = pd.read_csv('./datasets/concorrentes.csv', ';')

# pre processing
def get_dias_fechados():
    return ['2018-03-30', '2018-11-02', '2018-12-25', '2018-12-29', '2018-12-30', '2018-12-31', '2019-01-01', '2019-03-03', '2019-03-04', '2019-03-05', '2019-04-19', '2019-11-02', '2019-12-25', '2019-12-29', '2019-12-30', '2019-12-31', '2020-01-01', '2020-02-24', '2020-02-25']

def get_dias_fds():
    return ['Sunday', 'Saturday']

def get_dias_feriados():
    return ['2018-03-30', '2018-04-21', '2018-04-30', '2018-05-01', '2018-05-31', '2018-06-01', '2018-09-07', '2018-10-12', '2018-11-02', '2018-11-15', '2018-11-16', '2018-12-24', '2018-12-25', '2018-12-29', '2018-12-30', '2018-12-31', 
'2019-01-01', '2019-03-04', '2019-03-05', '2019-04-19', '2019-04-21', '2019-05-01', '2019-06-20', '2019-06-21', '2019-09-07', '2019-10-12', '2019-11-02', '2019-11-15', '2019-12-25', '2019-12-25', '2019-12-25', '2019-12-30', '2019-12-31', 
'2020-01-01', '2020-02-24', '2020-02-25']

def get_dias_festivos():
    return ['2018-05-13', '2018-08-12', '2018-12-25', '2018-12-29', '2018-12-30', '2018-12-31', '2019-01-01', '2019-03-03', '2019-03-04', '2019-03-05', '2019-05-12', '2019-08-11', '2019-12-25', '2019-12-29', '2019-12-30', '2019-12-31', '2020-01-01', '2020-02-24', '2020-02-25']

def get_vesperas_dias_festivos():
    vesperas = []
    for i in range(len(get_dias_festivos())):
        vesperas_dia_festivo_current = pd.Series(pd.date_range(end=get_dias_festivos()[i], periods=5, closed='left'))
        for j in range(len(vesperas_dia_festivo_current)):
            vesperas.append(vesperas_dia_festivo_current[j])
    return vesperas

def get_pos_dias_festivos():
    pos = []
    for i in range(len(get_dias_festivos())):
        pos_dia_festivo_current = pd.Series(pd.date_range(get_dias_festivos()[i], periods=3, closed='right'))
        for j in range(len(pos_dia_festivo_current)):
            pos.append(pos_dia_festivo_current[j])
    return pos

def get_dias_acerca_pagamento():
    return [5,6,7,8,9,10,28,29,30,31]

def get_meses_baixas_temporadas():
    return [1, 2]

# featured engineering
df.insert(loc=0, column='DATA', value=(pd.to_datetime(df_vendas['DATA']).dt.strftime("%Y%m%d")))
df.insert(loc=1, column='VENDAS', value=(df_vendas['VENDAS'].fillna(0).astype(int)))
df.insert(loc=2, column='IS_FECHADO', value=(pd.DatetimeIndex(df_temp['DATA']).isin(get_dias_fechados()).astype(int)))
df.insert(loc=3, column='IS_FDS', value=(pd.DatetimeIndex(df_temp['DATA']).day_name().isin(get_dias_fds())).astype(int))
df.insert(loc=4, column='IS_FERIADO', value=(pd.DatetimeIndex(df_temp['DATA']).isin(get_dias_feriados())).astype(int))
df.insert(loc=5, column='IS_DATA_FESTIVA', value=(pd.DatetimeIndex(df_temp['DATA']).isin(get_dias_festivos())).astype(int))
df.insert(loc=6, column='IS_VESPERA_DATA_FESTIVA', value=(pd.DatetimeIndex(df_temp['DATA']).isin(get_vesperas_dias_festivos())).astype(int))
df.insert(loc=7, column='IS_POS_DATA_FESTIVA', value=(pd.DatetimeIndex(df_temp['DATA']).isin(get_pos_dias_festivos())).astype(int))
df.insert(loc=8, column='IS_SEMANA_PAGAMENTO', value=(pd.DatetimeIndex(df_temp['DATA']).day.astype(int).isin(get_dias_acerca_pagamento()).astype(int)))
df.insert(loc=9, column='IS_BAIXA_TEMPORADA', value=(pd.DatetimeIndex(df_temp['DATA']).month.isin(get_meses_baixas_temporadas()).astype(int)))
df.insert(loc=10, column='QTD_CONCORRENTES', value=(df_conc['CONCORRENTES'].fillna(0).astype(int)))
df.insert(loc=11, column='PRECIPITACAO', value=(df_temp['PRECIPITACAO']))
df.insert(loc=12, column='TEMPERATURA', value=(df_temp['TEMPERATURA']))
df.insert(loc=13, column='UMIDADE', value=(df_temp['UMIDADE']))

# input & output
X = df.drop(columns=['DATA', 'VENDAS'])
# y = df.drop(columns=['DATA', 'IS_FECHADO', 'IS_FDS', 'IS_FERIADO', 'IS_DATA_FESTIVA', 'IS_VESPERA_DATA_FESTIVA', 'IS_POS_DATA_FESTIVA', 'IS_SEMANA_PAGAMENTO', 'IS_BAIXA_TEMPORADA', 'QTD_CONCORRENTES', 'PRECIPITACAO', 'TEMPERATURA', 'UMIDADE'])
X = df.drop(columns=['VENDAS'])
y = df.drop(columns=['IS_FECHADO', 'IS_FDS', 'IS_FERIADO', 'IS_DATA_FESTIVA', 'IS_VESPERA_DATA_FESTIVA', 'IS_POS_DATA_FESTIVA', 'IS_SEMANA_PAGAMENTO', 'IS_BAIXA_TEMPORADA', 'QTD_CONCORRENTES', 'PRECIPITACAO', 'TEMPERATURA', 'UMIDADE'])

# standardization & normalization
X = preprocessing.scale(X)

# shuffled and splitted into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

# training the model and get its score - LinearRegression
model = LinearRegression(n_jobs=-1) 
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(f'Acurácia do LinearRegression: {round(accuracy*100, 2)}%')

# training the model and get its score - Support Vector Regression
model2 = svm.SVR(kernel='poly')
model2.fit(X_train, y_train)
accuracy2 = model2.score(X_test, y_test)
print(f'Acurácia do Vector Regression: {round(accuracy2*100, 2)}%')

# training the model and get its score - DecisionTreeClassifier
model3 = DecisionTreeClassifier(criterion = 'entropy', random_state = 42)
model3.fit(X_train, y_train)
accuracy3 = model3.score(X_test, y_test)
print(f'Acurácia do DecisionTreeClassifier: {round(accuracy3*100, 2)}%')

# training the model and get its score - DecisionTreeRegressor
model4 = DecisionTreeRegressor(random_state=0)
model4.fit(X_train, y_train)
accuracy4 = model4.score(X_test, y_test)
print(f'Acurácia do DecisionTreeRegressor: {round(accuracy4*100, 2)}%')

# training the model and get its score - AdaBoostRegressor
model5 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=14), n_estimators=300, learning_rate=2, loss='exponential')
model5.fit(X_train, y_train)
accuracy5 = model5.score(X_test, y_test)
print(f'Acurácia do AdaBoostRegressor: {round(accuracy5*100, 2)}%')

# training the model and get its score - RandomForestClassifier
model6 = RandomForestClassifier(criterion = 'entropy', random_state = 42)
model6.fit(X_train, y_train)
accuracy6 = model6.score(X_test, y_test)
print(f'Acurácia do RandomForestClassifier: {round(accuracy6*100, 2)}%')

# training the model and get its score - BayesianRidge
model7 = BayesianRidge(compute_score=True)
model7.fit(X_train, y_train)
accuracy7 = model7.score(X_test, y_test)
print(f'Acurácia do BayesianRidge: {round(accuracy7*100, 2)}%')

# create forecast column
# df.insert(loc=2, column='VENDAS_FORECAST', value=(np.nan))

# plot chart
df.plot(y='VENDAS', x='DATA', figsize=(13, 4), linestyle='solid', linewidth=1, markersize=2, style="-o")
plt.legend(loc=1)
plt.title('HISTÓRICO DE VENDAS')
plt.xlabel('DATA')
plt.ylabel('QUANTIDADE DE ALMOÇOS')
plt.show()

ValueError: bad input shape (526, 2)